In [3]:
### We want to see if it is downstream to any of the TORC, GTR, Pib2 proteins
### T tests across the different conditions will show significance of connections.
### Output: csv file with original data and a new sheet with p-values and adjusted p-values

import pandas as pd
import os
from scipy.stats import ttest_ind
import statsmodels.stats.multitest as multi
import numpy as np

In [4]:
# open and convert the file to a Dataframe
filename: str = 'C:/Users/amyfa/vstest/phosph_data_caplab/230403gtroub2expt_working.csv' # change this to the file path of original data
full_df = pd.read_csv(filename)

# Function for Welch's T test
def perform_welchs_ttest(rowdata: pd.Series, threshold: float = 0.05):
    '''
    Uses Welch's T test to determine protein downstream connections to Torc, Pib2, Gtr1/2.
    Returns connections and p-values.

    Args: 
    rowdata (pd.DataFrame): contains 4 treatment conditions in quadruplicate
        1. WT in normal SD media
        2. WT in Rapamycin treatment (turns off Torc)
        3. Pib2KO in SD media
        4. GTRKO in SD media
    threshold (float): Threshold for the t-test. Default is 0.05.

    Returns - 
    A 1-D list of pvalues with length 3. [torc pval, pib2 pval, gtr pval] 
    '''
    conditions  = [rowdata.loc["WT_SD_1":"GTRKO_SD_4"].iloc[i:i+4].astype(float) for i in range(0, 16, 4)]
    pvalues = [float(ttest_ind(conditions[0], conditions[i], equal_var=False).pvalue) for i in range(1, len(conditions))]
    return pvalues


# test on just one row
# one_row: pd.Series = full_df[127:128].iloc[0]
# perform_welchs_ttest(one_row, threshold = 0.05)

# Apply to DataFrame
full_df["p-value"] = full_df.apply(perform_welchs_ttest, threshold = 1, axis = 1)
full_df["p-value"]

c:\Users\amyfa\vstest\.venv\lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\amyfa\vstest\.venv\lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\amyfa\vstest\.venv\lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\amyfa\vstest\.venv\lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision 

0         [nan, 0.38033980838065384, 0.39100221895576953]
1       [0.003569213615907592, 0.08928284138200536, 0....
2       [0.007113112276283214, 0.35547447278235045, 0....
3       [0.003229997292927913, 0.4375272936547459, 0.8...
4       [0.010235693882455316, 0.2468912789787353, 0.5...
                              ...                        
7802    [0.9979544571304928, 0.16902918042857576, 0.00...
7803    [0.9993087004043089, 0.9821232544090529, 0.659...
7804    [0.9997529325106471, 0.225384522393724, 0.8757...
7805    [0.999958468036912, 0.9555001665539399, 0.9445...
7806    [0.9999215317049629, 0.11278348602623375, 0.16...
Name: p-value, Length: 7807, dtype: object

In [5]:
# collapse into 1D list
collapsed_pvals = [item in sublist for sublist in full_df["p-value"] for item in sublist]

# Adjust p-values
adjusted_pvals = multi.multipletests(pvals = collapsed_pvals, alpha = 0.05, method = "fdr_bh")

In [6]:
# Separate into three different tests
torc_pvals, pib2_pvals, gtr_pvals = list(zip(*full_df["p-value"]))

# Check properly unpacked
for i in range(0,3):
    print([pvals[i] for pvals in full_df["p-value"][0:5]])
    print(list(zip(*full_df["p-value"]))[i][0:5])
    print("\n")

# Adjust p-values

torc_adj = multi.multipletests(pvals = torc_pvals[1:], alpha = .05, method = "fdr_bh")[1] # remove the nan value
torc_adj = np.insert(torc_adj, 0, float('nan')) # and put it back in as nan
pib2_adj = multi.multipletests(pvals = pib2_pvals, alpha = .05, method = "fdr_bh")[1]
gtr_adj = multi.multipletests(pvals = pib2_pvals, alpha = 0.05, method = "fdr_bh")[1]
torc_adj, pib2_adj, gtr_adj

[nan, 0.003569213615907592, 0.007113112276283214, 0.003229997292927913, 0.010235693882455316]
(nan, 0.003569213615907592, 0.007113112276283214, 0.003229997292927913, 0.010235693882455316)


[0.38033980838065384, 0.08928284138200536, 0.35547447278235045, 0.4375272936547459, 0.2468912789787353]
(0.38033980838065384, 0.08928284138200536, 0.35547447278235045, 0.4375272936547459, 0.2468912789787353)


[0.39100221895576953, 0.3191296452833939, 0.8614726830808649, 0.8784051254484133, 0.502684910634774]
(0.39100221895576953, 0.3191296452833939, 0.8614726830808649, 0.8784051254484133, 0.502684910634774)




(array([       nan, 0.11560698, 0.14237168, ..., 0.99995847, 0.99995847,
        0.99995847]),
 array([0.96509927, 0.94165667, 0.96509927, ..., 0.9637096 , 0.99488884,
        0.95300272]),
 array([0.96509927, 0.94165667, 0.96509927, ..., 0.9637096 , 0.99488884,
        0.95300272]))

In [19]:
# Insert adjusted into dataframe
full_df["torc adj. pval"] = torc_adj
full_df["pib2 adj. pval"] = pib2_adj
full_df["gtr adj. pval"] = gtr_adj

# indicate significance with specific threshold
def determine_significance(rowdata: pd.Series, threshold = 0.05):
    is_sig = [pval < threshold for pval in rowdata.loc['torc adj. pval': 'gtr adj. pval']]
    return None if True not in is_sig else "significant"

# Testing on just one row
# one_row: pd.Series = full_df[5883:5884].iloc[0]
# print(one_row)
# print(determine_significance(one_row))


full_df["significance"] = full_df.apply(determine_significance, axis = 1)
full_df["significance"][5883]

# Create output csv
columns = list(full_df.columns[1:4]) + ["p-value", "torc adj. pval", "pib2 adj. pval", "gtr adj. pval", "significance"]
full_df.to_csv("C:/Users/amyfa/vstest/phosph_data_caplab/output.csv", columns = columns) # output location